<a href="https://colab.research.google.com/github/Silsl0/SertAI/blob/main/SertAI_Results_WebMap.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# SertAI — **Resultados & Web Map** (Folium)
Notebook simples: lê os **exports** (GeoJSON) e gera um **mapa interativo** com camadas (lagoas chuvoso/seco, vetores de migração) e **KPIs** (áreas e velocidades).

**Entrada (no Google Drive):**
- Pasta: `/content/drive/MyDrive/SertAI_GEE_Exports/`
- `Polygons_Lagoons_Wet.geojson`
- `Polygons_Lagoons_Dry.geojson`
- `offset_outputs/offset_vectors_lines.geojson` *(ou)* `offset_outputs/offset_vectors.geojson`

**Saída:**
- `map_resultados.html` (na mesma pasta)
- `summary_results.json` (KPIs numéricos)


In [34]:
%pip install -q folium geopandas shapely pyproj mapclassify

In [35]:

# ==== Parâmetros ====
DRIVE_DIR = '/content/drive/MyDrive/SertAI_GEE_Exports'
WET_GJ    = f'{DRIVE_DIR}/Polygons_Lagoons_Wet.geojson'
DRY_GJ    = f'{DRIVE_DIR}/Polygons_Lagoons_Dry.geojson'
LINES_GJ  = f'{DRIVE_DIR}/offset_outputs/offset_vectors_lines.geojson'
POINTS_GJ = f'{DRIVE_DIR}/offset_outputs/offset_vectors.geojson'  # fallback se LINES não existir
OUT_HTML  = f'{DRIVE_DIR}/map_resultados.html'
OUT_JSON  = f'{DRIVE_DIR}/summary_results.json'

# Se não tiver linhas, criar linhas curtas a partir dos pontos (dx_m,dy_m)
MAKE_LINES_FROM_POINTS = True
WET_DATE = '2024-05-15'   # apenas para legenda/resumo
DRY_DATE = '2024-09-15'


In [36]:

import sys, os
if 'google.colab' in sys.modules:
    from google.colab import drive
    drive.mount('/content/drive', force_remount=True)

print('Arquivos:')
for p in [WET_GJ, DRY_GJ, LINES_GJ, POINTS_GJ]:
    print(os.path.exists(p), p)


Mounted at /content/drive
Arquivos:
True /content/drive/MyDrive/SertAI_GEE_Exports/Polygons_Lagoons_Wet.geojson
True /content/drive/MyDrive/SertAI_GEE_Exports/Polygons_Lagoons_Dry.geojson
False /content/drive/MyDrive/SertAI_GEE_Exports/offset_outputs/offset_vectors_lines.geojson
True /content/drive/MyDrive/SertAI_GEE_Exports/offset_outputs/offset_vectors.geojson


In [37]:

import geopandas as gpd
from shapely.geometry import LineString, Point
import numpy as np
import json

# Carregar lagoas
wet = gpd.read_file(WET_GJ)
dry = gpd.read_file(DRY_GJ)

# CRS seguro para áreas (UTM pela longitude do centróide das lagoas)
def auto_utm_crs(gdf):
    c = gdf.to_crs(4326).geometry.unary_union.centroid
    lon, lat = c.x, c.y
    zone = int((lon + 180)//6) + 1
    south = "+south" if lat < 0 else ""
    return f"+proj=utm +zone={zone} {south} +datum=WGS84 +units=m +no_defs"

utm = auto_utm_crs(wet)

wet_m = wet.to_crs(utm); dry_m = dry.to_crs(utm)
wet_area_km2 = float(wet_m.area.sum()/1e6)
dry_area_km2 = float(dry_m.area.sum()/1e6)
delta_km2 = wet_area_km2 - dry_area_km2

# Carregar vetores (linhas)
lines = None
if os.path.exists(LINES_GJ):
    lines = gpd.read_file(LINES_GJ)
elif os.path.exists(POINTS_GJ) and MAKE_LINES_FROM_POINTS:
    pts = gpd.read_file(POINTS_GJ)
    # criar linhas curtas (F=5 para visualizar)
    F = 5.0
    if 'dx_m' in pts.columns and 'dy_m' in pts.columns:
        geoms = []
        for _, r in pts.iterrows():
            if r.geometry is None or not isinstance(r.geometry, Point):
                continue
            x, y = r.geometry.x, r.geometry.y
            geoms.append(LineString([(x, y), (x + F*float(r.get('dx_m',0)), y + F*float(r.get('dy_m',0)))]))
        keep_cols = [c for c in pts.columns if c in ('speed_m_per_year','dir_deg_math','error')]
        lines = gpd.GeoDataFrame(pts[keep_cols].copy(), geometry=geoms, crs=pts.crs)
    else:
        # sem dx/dy, apenas pontos
        lines = None

# KPIs de velocidade (se existirem)
kpis = {}
if lines is not None and 'speed_m_per_year' in lines.columns:
    sp = lines['speed_m_per_year'].astype(float).replace([np.inf,-np.inf], np.nan).dropna()
    if len(sp) > 0:
        kpis['speed_p50'] = float(sp.quantile(0.50))
        kpis['speed_p90'] = float(sp.quantile(0.90))
        # direção média (bearing 0°=N) a partir de dx/dy se existirem, senão usar dir_deg_math
        if {'dx_m','dy_m'}.issubset(lines.columns):
            dx = lines['dx_m'].astype(float); dy = lines['dy_m'].astype(float)
            mean_dir = float(np.degrees(np.arctan2(dy.mean(), dx.mean())))
        elif 'dir_deg_math' in lines.columns:
            mean_dir = float(np.nanmean(lines['dir_deg_math'].astype(float)))
        else:
            mean_dir = np.nan
        mean_bearing = (90 - mean_dir) % 360 if np.isfinite(mean_dir) else np.nan
        kpis['mean_bearing_deg'] = float(mean_bearing) if np.isfinite(mean_bearing) else None

kpis['wet_area_km2'] = wet_area_km2
kpis['dry_area_km2'] = dry_area_km2
kpis['delta_km2']    = delta_km2
kpis['period'] = f"{WET_DATE} → {DRY_DATE}"

with open(OUT_JSON, 'w') as f:
    json.dump(kpis, f, indent=2)

print('KPIs:', json.dumps(kpis, indent=2))


/tmp/ipython-input-677975485.py:12: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  c = gdf.to_crs(4326).geometry.unary_union.centroid


KPIs: {
  "speed_p50": 0.0,
  "speed_p90": 29.64876572745471,
  "mean_bearing_deg": 91.70956100917904,
  "wet_area_km2": 623.3762887369749,
  "dry_area_km2": 453.91479669836997,
  "delta_km2": 169.4614920386049,
  "period": "2024-05-15 \u2192 2024-09-15"
}


In [38]:
# ======================
# Final Web Map (Folium) — English UI
# ======================
import folium
from folium.features import GeoJsonTooltip
from folium.plugins import Fullscreen, MiniMap, MeasureControl, MousePosition
import numpy as np
import os, json

# ---- visual params ----
BINS = [20, 40, 80, 120]                 # fixed classes (m/yr): ≤20, ≤40, ≤80, ≤120, >
COLS = ['#a6cee3', '#1f78b4', '#33a02c', '#fb9a99', '#e31a1c']
FAST_TH = 20                              # “Fast vectors” layer: > FAST_TH m/yr

# performance
SIMPLIFY_LINES = True
SIMPL_TOL_M = 5.0                         # simplification tolerance (m)

def color_by_speed(v, qs=BINS, cols=COLS):
    try:
        v = float(v)
    except:
        return cols[0]
    for q, c in zip(qs, cols):
        if v <= q:
            return c
    return cols[-1]

def auto_utm_crs(gdf):
    g = gdf.to_crs(4326).geometry
    c = (g.union_all() if hasattr(g, "union_all") else g.unary_union).centroid
    lon, lat = float(c.x), float(c.y)
    zone = int((lon + 180)//6) + 1
    south = "+south" if lat < 0 else ""
    return f"+proj=utm +zone={zone} {south} +datum=WGS84 +units=m +no_defs"

def bearing_to_compass(b):
    dirs = ["N","NNE","NE","ENE","E","ESE","SE","SSE",
            "S","SSW","SW","WSW","W","WNW","NW","NNW"]
    try:
        return dirs[int((float(b) % 360)/22.5 + 0.5) % 16]
    except:
        return "–"

# ---- lagoons in WGS84 ----
wet_ll = wet.to_crs(4326)
dry_ll = dry.to_crs(4326)
geom_u = wet_ll.geometry.union_all() if hasattr(wet_ll.geometry, "union_all") else wet_ll.geometry.unary_union
ctr = geom_u.centroid

# ---- base map ----
m = folium.Map(location=[float(ctr.y), float(ctr.x)], zoom_start=11, tiles='cartodbpositron')
folium.TileLayer(
    tiles='https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
    name='ESRI World Imagery',
    attr='Esri'
).add_to(m)

# ---- lagoons (styled) ----
# Wet: outline + light fill
folium.GeoJson(
    wet_ll, name='Lagoons — Wet season',
    style_function=lambda f: {
        'color': '#1f78b4',
        'weight': 1,
        'fillColor': '#1f78b4',
        'fillOpacity': 0.15
    }
).add_to(m)

# Dry: filled
folium.GeoJson(
    dry_ll, name='Lagoons — Dry season',
    style_function=lambda f: {
        'color': '#ffbf00',
        'weight': 1,
        'fillColor': '#ffbf00',
        'fillOpacity': 0.25
    }
).add_to(m)

# ---- vectors (lines) ----
if (lines is not None) and (not lines.empty):
    L = lines.dropna(subset=['geometry']).copy()

    # simplify while keeping attributes
    if SIMPLIFY_LINES and not L.empty:
        try:
            utm = auto_utm_crs(wet)
            L = L.to_crs(utm).copy()
            L['geometry'] = L.geometry.simplify(SIMPL_TOL_M, preserve_topology=True)
            L = L[~L.geometry.is_empty & L.geometry.notna()].copy()
            L = L.to_crs(4326)
        except Exception:
            L = L.to_crs(4326)
    else:
        L = L.to_crs(4326)

    cols_avail = list(getattr(L, 'columns', []))
    tooltip_fields = [c for c in ('speed_m_per_year','dir_deg_math','error') if c in cols_avail]

    def style_fun(feat):
        v = feat['properties'].get('speed_m_per_year', None)
        return {'color': color_by_speed(v), 'weight': 2, 'opacity': 0.9}

    folium.GeoJson(
        L, name='Offset vectors', style_function=style_fun,
        tooltip=GeoJsonTooltip(fields=tooltip_fields,
                               aliases=['speed (m/yr)','dir (deg)','error'][:len(tooltip_fields)])
    ).add_to(m)

    # “Fast vectors” (> FAST_TH m/yr)
    if 'speed_m_per_year' in cols_avail:
        try:
            Lfast = L[L['speed_m_per_year'].astype(float) > FAST_TH]
        except Exception:
            Lfast = L.iloc[0:0]
        if not Lfast.empty:
            folium.GeoJson(
                Lfast, name=f'Vectors (> {FAST_TH} m/yr)',
                style_function=lambda f: {'color':'#e31a1c','weight':2,'opacity':0.9},
                tooltip=GeoJsonTooltip(fields=tooltip_fields,
                                       aliases=['speed (m/yr)','dir (deg)','error'][:len(tooltip_fields)])
            ).add_to(m)

# ---- plugins ----
Fullscreen().add_to(m)
MiniMap(toggle_display=True).add_to(m)

# Put MeasureControl *near cartodbpositron/LayerControl* (bottom-left),
# and use km / km² with secondary m / ha.
m.add_child(MeasureControl(
    position='bottomleft',
    primary_length_unit='kilometers',
    secondary_length_unit='meters',
    primary_area_unit='sqkilometers',
    secondary_area_unit='hectares',
    active_color='#e67e22',
    completed_color='#2c3e50'
))
MousePosition(prefix='lat/lon').add_to(m)

# LayerControl bottom-left (keeps top-right free for the cards)
folium.LayerControl(collapsed=False, position='bottomleft').add_to(m)

# ---- cards (top-right): Results + Speed legend side-by-side ----
# Larger, clearer text
BASE_CARD_CSS = "background:white;padding:12px 14px;border:1px solid #bbb;font-size:14px;line-height:1.35;"
TITLE_CSS = "font-weight:600;font-size:14px;"

# speed legend
bins_labels = [f'≤{q:.0f}' for q in BINS] + ['>']
legend_html = f'<div style="{BASE_CARD_CSS}">'
legend_html += f'<div style="{TITLE_CSS}">Dune migration speed (m/yr)</div>'
for label, c in zip(bins_labels, COLS + [COLS[-1]]):
    legend_html += f'<div><i style="background:{c};width:12px;height:12px;display:inline-block;margin-right:8px;border:1px solid #888;"></i>{label}</div>'
legend_html += '</div>'

# results card (optional)
KPIS = f"{DRIVE_DIR}/summary_results.json"
card_html = f"""
    <div style="{BASE_CARD_CSS}">
      <div style="{TITLE_CSS}">SertAI — Results (SAR / AOT)</div>
      <div>Period: {k.get('period','–')}</div>
      <hr style="margin: 6px 0; border-top: 1px solid #ddd;">

      <div style="{TITLE_CSS}; color:#1f78b4;">Hydrology (Lagoon)</div>
      <div>Lagoon area — wet season: {k.get('wet_area_km2',0):,.1f} km²</div>
      <div>Lagoon area — dry season: {k.get('dry_area_km2',0):,.1f} km²</div>
      <div style="font-weight:600;">Δ Lagoon Area: {k.get('delta_km2',0):+,.1f} km²</div>

      <hr style="margin: 6px 0; border-top: 1px solid #ddd;">

      <div style="{TITLE_CSS}; color:#e31a1c;">Aeolian (Dune Migration)</div>
      <div>Migration Speed (90% of Active Dunes): <span style="font-weight:600;">{k.get('speed_p90',0):,.0f} m/yr</span></div>
      <div>Overall Motion Speed (Median): {k.get('speed_p50',0):,.0f} m/yr</div>
      <div>Mean Migration Direction: <span style="font-weight:600;">{bearing_str}</span></div>
    </div>"""

# container (top-right), side-by-side with some gap
ui_stack = f"""
<div style="position: fixed; top: 20px; right: 20px; z-index: 9999; display: flex; gap: 14px;">
  {card_html if card_html else ""}
  {legend_html}
</div>
"""
m.get_root().html.add_child(folium.Element(ui_stack))

# (optional) small CSS tweak to keep measure panel readable
custom_css = """
<style>
.leaflet-control-measure .results .stats,
.leaflet-control-measure .results .coord,
.leaflet-control-measure .toggle { font-size: 13px; }
</style>
"""
m.get_root().html.add_child(folium.Element(custom_css))

# ---- save ----
m.save(OUT_HTML)
OUT_HTML



'/content/drive/MyDrive/SertAI_GEE_Exports/map_resultados.html'

In [39]:
from IPython.display import IFrame
IFrame('/content/drive/MyDrive/SertAI_GEE_Exports/map_resultados.html', width=1100, height=650)
